In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

In [2]:
walmart = pd.read_csv('walmart.csv')
target = pd.read_csv('target.csv')
amazon = pd.read_csv('amazon.csv')
costco = pd.read_csv('costco.csv')
kroger = pd.read_csv('kroger.csv')

In [3]:
df = pd.concat(map(pd.read_csv, ['walmart.csv', 'target.csv','amazon.csv','costco.csv','kroger.csv']),ignore_index = True)

In [4]:
df.shape

(3715, 3)

In [5]:
df.loc[:,'tokens_final'] = df.loc[:,'tokens_final'].apply(lambda x: literal_eval(x))

In [6]:
import gensim
from gensim.utils import simple_preprocess
from gensim.test.utils import common_corpus, common_dictionary

import pprint
pp = pprint.PrettyPrinter(indent=4)

In [7]:
dictionary = gensim.corpora.Dictionary(df["tokens_final"])

dictionary.filter_extremes(no_below=1, no_above=0.5) #, keep_n=100000)

bow_corpus = [dictionary.doc2bow(word) for word in df["tokens_final"]]

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=15, id2word=dictionary, 
                                       passes=2, workers=2, chunksize=100, random_state=1000)

In [8]:
pp.pprint(lda_model.print_topics())

[   (   0,
        '0.057*"unit" + 0.053*"amount" + 0.047*"value" + 0.043*"performance" + '
        '0.032*"participant" + 0.031*"restrict" + 0.031*"award" + 0.024*"vest" '
        '+ 0.024*"interest" + 0.024*"cash"'),
    (   1,
        '0.062*"kroger" + 0.035*"award" + 0.029*"section" + 0.020*"time" + '
        '0.018*"agreement" + 0.016*"condition" + 0.015*"term" + 0.014*"mean" + '
        '0.014*"participant" + 0.014*"employee"'),
    (   2,
        '0.042*"ownership" + 0.041*"include" + 0.039*"group" + 0.037*"power" + '
        '0.034*"officer" + 0.023*"hold" + 0.021*"person" + 0.018*"information" '
        '+ 0.016*"owner" + 0.016*"inc."'),
    (   3,
        '0.048*"president" + 0.040*"serve" + 0.034*"experience" + '
        '0.032*"chairman" + 0.028*"vice" + 0.028*"officer" + 0.023*"business" '
        '+ 0.022*"inc." + 0.020*"service" + 0.019*"corporation"'),
    (   4,
        '0.027*"cost" + 0.025*"asset" + 0.021*"rate" + 0.020*"sale" + '
        '0.017*"store" + 0.017*"expe

In [9]:
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

In [10]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.105497 -0.147232       1        1  13.256654
4      0.054132 -0.150455       2        1  11.126772
9      0.148099 -0.142033       3        1   9.281467
3     -0.192455 -0.210747       4        1   7.558033
6      0.195205  0.154327       5        1   6.773318
5      0.177755 -0.016149       6        1   6.470808
14     0.152078 -0.032438       7        1   6.380615
11    -0.178124  0.214503       8        1   6.282763
0      0.271935  0.013643       9        1   5.815441
1      0.150720  0.113582      10        1   5.472498
13    -0.232165 -0.043742      11        1   5.403916
8     -0.037038  0.032179      12        1   4.996787
12    -0.183898  0.230490      13        1   3.845350
10    -0.115325  0.011246      14        1   3.822639
2     -0.105425 -0.027173      15        1   3.512946, topic_info=             Term         Freq        Total Category  logprob  loglift
4056  performance  6121.000000  6121.000000  Default  30.0000  30.0000
1132       kroger  4705.000000  4705.000000  Default  29.0000  29.0000
120        option  4329.000000  4329.000000  Default  28.0000  28.0000
89          audit  3426.000000  3426.000000  Default  27.0000  27.0000
90          award  5730.000000  5730.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
964        target   217.802902  2646.716309  Topic15  -4.5381   0.8512
252        number   224.372528  3119.684570  Topic15  -4.5084   0.7165
342          base   238.908188  4246.946289  Topic15  -4.4456   0.4708
64         report   211.014618  3756.200684  Topic15  -4.5697   0.4695
71       security   196.956207  2056.624268  Topic15  -4.6387   1.0029

[1032 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
86       12  0.814446  abbreviation
86       13  0.081445  abbreviation
224      13  0.994913       abstain
225       8  0.007616    abstention
225      13  0.986243    abstention
...     ...       ...           ...
2264     12  0.019094         yahoo
2264     15  0.019094         yahoo
2194      7  0.969146          zone
2194     15  0.019778          zone
2819     13  0.755373         •vote

[4216 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 10, 4, 7, 6, 15, 12, 1, 2, 14, 9, 13, 11, 3])